Окружение для Colab

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Импорт необходимых библиотек


In [ ]:
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, datediff
from pyspark.sql import functions as F

In [ ]:
spark = SparkSession.builder.appName('PySparkTasks').getOrCreate()

Загружаем данные

In [ ]:
df = spark.read.parquet('/content/drive/My Drive/Data for Collab/PySpark/clickstream.parquet')

NameError: ignored

In [ ]:
def is_cpc_cpm(df):
  '''Добавляем признаки CPC\CPM.
     is_cpm: 1 если тип объявления CPM, иначе 0.
     is_cpc: 1 если тип объявления CPC, иначе 0.
  '''
  df = df.withColumn('is_cpc',F.when(df.ad_cost_type == 'CPC', 1).otherwise(0))
  df = df.withColumn('is_cpm',F.when(df.ad_cost_type == 'CPM', 1).otherwise(0))
  return df

In [ ]:
df = is_cpc_cpm(df)

In [ ]:
df.show(2)

+----------+-------------------+-----+--------+------+---------------+-----------------+------------+-------+---------+---------------------+------+------+
|      date|               time|event|platform| ad_id|client_union_id|compaign_union_id|ad_cost_type|ad_cost|has_video|target_audience_count|is_cpc|is_cpm|
+----------+-------------------+-----+--------+------+---------------+-----------------+------------+-------+---------+---------------------+------+------+
|2019-04-01|2019-03-31 21:00:48| view| android| 45061|          34734|            45061|         CPM|  200.6|        0|              1955269|     0|     1|
|2019-04-01|2019-03-31 21:00:48| view|     web|121288|         121288|           121288|         CPM|  187.4|        0|               232011|     0|     1|
+----------+-------------------+-----+--------+------+---------------+-----------------+------------+-------+---------+---------------------+------+------+
only showing top 2 rows



In [ ]:
def ctr_adding(df):
  '''Добавление колонки CTR.
    CTR равен отношению числа кликов к числу просмотров.
  '''
  #временно добавим колонки с признаком клика или просмотра
  df = df.withColumn('click',F.when(df.event == 'click', 1).otherwise(0))
  df = df.withColumn('view',F.when(df.event == 'view', 1).otherwise(0))

  #сгруппируем по ad_id, просуммируем клики и просмотры для дальнейшего расчета CTR
  gdf = df.groupby('ad_id')\
  .agg({'click':'sum','view':'sum'})\
  .withColumnRenamed('sum(click)','clicks')\
  .withColumnRenamed('sum(view)', 'views')

  #рассчитаем ctr
  gdf = gdf.withColumn('ctr', F.round(gdf.clicks/gdf.views,3)*100)

  #добавим ctr в таблицу
  df = df.join(gdf, on='ad_id')

  return df

In [ ]:
df = ctr_adding(df)

In [ ]:
df.show(2)

+------+----------+-------------------+-----+--------+---------------+-----------------+------------+-------+---------+---------------------+------+------+-----+----+-----+------+---+
| ad_id|      date|               time|event|platform|client_union_id|compaign_union_id|ad_cost_type|ad_cost|has_video|target_audience_count|is_cpc|is_cpm|click|view|views|clicks|ctr|
+------+----------+-------------------+-----+--------+---------------+-----------------+------------+-------+---------+---------------------+------+------+-----+----+-----+------+---+
| 45061|2019-04-01|2019-03-31 21:00:48| view| android|          34734|            45061|         CPM|  200.6|        0|              1955269|     0|     1|    0|   1|  795|     6|0.8|
|121288|2019-04-01|2019-03-31 21:00:48| view|     web|         121288|           121288|         CPM|  187.4|        0|               232011|     0|     1|    0|   1|  583|     0|0.0|
+------+----------+-------------------+-----+--------+---------------+----------

In [ ]:
def day_count_adding(df):
  '''Добавляем колонку day_count.
    Это число дней, которое показывалась реклама.
  '''
  #находим разность между текущей датой и датой объявления
  df = df.withColumn('day_count', datediff(F.current_date(),df.date))
  return df

In [ ]:
df = day_count_adding(df)

In [ ]:
df.show(2)

+------+----------+-------------------+-----+--------+---------------+-----------------+------------+-------+---------+---------------------+------+------+-----+----+-----+------+---+---------+
| ad_id|      date|               time|event|platform|client_union_id|compaign_union_id|ad_cost_type|ad_cost|has_video|target_audience_count|is_cpc|is_cpm|click|view|views|clicks|ctr|day_count|
+------+----------+-------------------+-----+--------+---------------+-----------------+------------+-------+---------+---------------------+------+------+-----+----+-----+------+---+---------+
| 45061|2019-04-01|2019-03-31 21:00:48| view| android|          34734|            45061|         CPM|  200.6|        0|              1955269|     0|     1|    0|   1|  795|     6|0.8|      969|
|121288|2019-04-01|2019-03-31 21:00:48| view|     web|         121288|           121288|         CPM|  187.4|        0|               232011|     0|     1|    0|   1|  583|     0|0.0|      969|
+------+----------+-----------

In [ ]:
def cols_deleting(df):
  '''Удаляем ненужные колонки из таблицы'''
  cols = ['ad_cost_type','event','client_union_id','platform','click','view','views','clicks', 'time', 'date', 'compaign_union_id']
  #выбросим ненужные колонки
  df = df.drop(*cols)
  return df

In [ ]:
df = cols_deleting(df)

In [ ]:
def df_splitting(df):
  '''Разделяем данные в соотношении 0.75 к 0.25'''
  train, test = df.randomSplit([0.75,0.25], 17)
  return train, test

In [ ]:
train, test = df_splitting(df)

In [ ]:
def writing_to_file(train, test):
  '''Записываем наборы данных в отдельные файлы'''
  test.coalesce(1).write.parquet('result/test/test.parquet')
  train.coalesce(1).write.parquet('result/train/train.parquet')

In [ ]:
writing_to_file(train, test)

In [ ]:
#arg=sys.argv[1:] - пропускаем название скрипта
#train.coalesce(1).write.parquet('{}/train'.format(arg[1])) - второй элемент в спсике arg - это название конечной папки

In [ ]:
a='res_1'

In [ ]:
test.coalesce(1).write.parquet('{}/abc'.format('res_2'))